In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from matplotlib.colors import SymLogNorm

from IPython.display import display
from lib.util_plot import *
from lib.constants import *
from lib.multilevel_analysis import *
from lib.pandas_impl import *
from lib.pandas_stats_VIS import *
from lib.matrix_analyzer import *
from lib.pandas_compute import *

import lib.VIS_L23_preprocessing.vis_L23_constants as VIS

In [ ]:
data_folder = Path.cwd() / 'data' / 'VIS'
eval_folder_5 = Path.cwd() / 'data' / 'eval' / 'VIS_24-12-17_5mu'
eval_folder_10 = Path.cwd() / 'data' / 'eval' / 'VIS_24-12-17_10mu'
eval_folder_25 = Path.cwd() / 'data' / 'eval' / 'VIS_24-12-17_25mu'
plot_folder = eval_folder_5 / "plots"
plot_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
def load_stats(eval_folder):
    filename = eval_folder / "multilevel_analysis.pkl"
    with open(filename, 'rb') as file:
        multilevel_analysis = pickle.load(file)

    return multilevel_analysis.stats, multilevel_analysis.df_summary

stats_5, df_summary_5 = load_stats(eval_folder_5)
stats_10, df_summary_10 = load_stats(eval_folder_10)
stats_25, df_summary_25 = load_stats(eval_folder_25)

In [ ]:
def stirling_ln_factorial(n):
    return n * np.log(n) - n + 0.5 * np.log(2 * np.pi * n)

def get_ratio(a_log, b_log):
    return np.exp(a_log - b_log)

def print_stats(df_summary):
    df_volume = df_summary.groupby("overlap_volume").agg({"empirical" : "sum"})

    print("num synapses", df_volume.sum().values[0])
    print("num cubes", len(df_volume))    
    print("mean syn per cube", df_volume.mean().values[0])
    print("median/25th/75th percentile", np.quantile(df_volume.empirical, [0.5, 0.25, 0.75]))

In [ ]:
print_stats(df_summary_5)

In [ ]:
labels = [STR_EMPIRICAL, STR_NULL + r" (25$\mu m$)", STR_NULL + r" (10$\mu m$)", STR_NULL + r" (5$\mu m$)"]

initPlotSettings(False)

COLORS_CATEGORICAL_MUTED = sns.color_palette("muted") 
COLORS_CATEGORICAL_DARK = sns.color_palette("dark") 
COLORS_CATEGORICAL_COLORBLIND = sns.color_palette("colorblind") 
COLORS_CATEGORICAL_BRIGHT = sns.color_palette("bright") 
COLORS_CATEGORICAL_PASTEL = sns.color_palette("pastel") 

C_EMPIRICAL = COLORS_CATEGORICAL[2]
C_M1 = COLORS_CATEGORICAL_MUTED[7]
C_M2 = COLORS_CATEGORICAL_BRIGHT[7]
C_M3 = COLORS_CATEGORICAL_PASTEL[7]

colors = [C_EMPIRICAL, C_M1, C_M2, C_M3]

In [ ]:
datasets = [stats_5.to_numpy(SELECTION_CELLTYPE, EMPIRICAL), 
            stats_25.to_numpy(SELECTION_CELLTYPE, MODEL_NULL),
            stats_10.to_numpy(SELECTION_CELLTYPE, MODEL_NULL),
            stats_5.to_numpy(SELECTION_CELLTYPE, MODEL_NULL)]

img = plot_bar_chart(datasets,
                     dataset_labels = labels, 
                     colors=colors,
                     x_labels=SELECTION_CELLTYPE,
                     y_lim=(10, 10**4), use_log=True, error_bars=True,
                     adjust_left=0.2, fig_size=figsize_mm_to_inch(50,40), 
                     filename=plot_folder/f"VIS_overlap_comparison_syncount.png")
display(img)

In [ ]:
datasets = [stats_5.to_numpy(SELECTION_EXC_SUBCELLULAR, EMPIRICAL), 
            stats_25.to_numpy(SELECTION_EXC_SUBCELLULAR, MODEL_NULL),
            stats_10.to_numpy(SELECTION_EXC_SUBCELLULAR, MODEL_NULL),
            stats_5.to_numpy(SELECTION_EXC_SUBCELLULAR, MODEL_NULL)]

img = plot_bar_chart(datasets,
                     #dataset_labels = labels, 
                     colors=colors,
                     x_labels=SELECTION_EXC_SUBCELLULAR,
                     y_lim=(10, 10**4), use_log=True, error_bars=True,
                     adjust_left=0.2, fig_size=figsize_mm_to_inch(60,40), 
                     filename=plot_folder/f"VIS_overlap_comparison_syncount_EXC_subcellular.png")
display(img)

In [ ]:
datasets = [stats_5.to_numpy(SELECTION_INH_SUBCELLULAR, EMPIRICAL), 
            stats_25.to_numpy(SELECTION_INH_SUBCELLULAR, MODEL_NULL),
            stats_10.to_numpy(SELECTION_INH_SUBCELLULAR, MODEL_NULL),
            stats_5.to_numpy(SELECTION_INH_SUBCELLULAR, MODEL_NULL)]

img = plot_bar_chart(datasets,
                     #dataset_labels = labels, 
                     colors=colors,
                     x_labels=SELECTION_INH_SUBCELLULAR,
                     y_lim=(10, 10**4), use_log=True, error_bars=True,
                     adjust_left=0.2, fig_size=figsize_mm_to_inch(60,40), 
                     filename=plot_folder/f"VIS_overlap_comparison_syncount_INH_subcellular.png")
display(img)

In [ ]:
datasets = [stats_5.to_numpy(SELECTION_CLUSTER_8, EMPIRICAL), 
            stats_25.to_numpy(SELECTION_CLUSTER_8, MODEL_NULL),
            stats_10.to_numpy(SELECTION_CLUSTER_8, MODEL_NULL),
            stats_5.to_numpy(SELECTION_CLUSTER_8, MODEL_NULL)]

x_labels = [l.split("-")[1] for l in SELECTION_CLUSTER_8]
        
        
filename = plot_folder/f"VIS_overlap_comparison_{SYNCLUSTERS}_0-8.png"
img = plot_line_chart(datasets, x_labels=x_labels, linestyles=[".-",".-",".-",".-"],
                      colors=colors, y_lim=(0, 10**6),       
    dataset_labels=labels, error_bars=True, use_log=True, hline_y=1,  adjust_left=0.2, adjust_bottom=0.22, fig_size=figsize_mm_to_inch(60,40), filename=filename)
display(img)

In [ ]:

datasets = [stats_5.to_numpy(SELECTION_MOTIF, EMPIRICAL), 
            stats_25.to_numpy(SELECTION_MOTIF, MODEL_NULL),
            stats_10.to_numpy(SELECTION_MOTIF, MODEL_NULL),
            stats_5.to_numpy(SELECTION_MOTIF, MODEL_NULL)]

filename = plot_folder/f"VIS_overlap_comparison_{MOTIFS}.png"
img = plot_motifs_bar_chart(datasets, 
    colors = colors,
    dataset_labels=labels, quantile_low=25, quantile_high=75,
    use_mean=False, marker_size=4, capsize=0, y_axis_label="occurrences relative to random",
    error_bars=True, y_lim=(-0.5,10**6), use_log=True, fig_size=figsize_mm_to_inch(160,50), adjust_left=0.07, filename=filename)
display(img)